In [1]:
!pip install tensorflow==2.5.0  opencv-python matplotlib

In [2]:
import cv2
import matplotlib.pyplot as plt
import os 
import random
import numpy as np
import uuid
import threading

In [3]:
#functional api

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer,Conv2D,Dense,MaxPooling2D,Input,Flatten
import tensorflow as tf

2023-04-09 23:36:36.006489: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/priyanshu/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-04-09 23:36:36.006507: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## creating folder structures  using os library ##

In [4]:
pos_path=os.path.join('data','positive')
neg_path=os.path.join('data','negative')
anchor_path=os.path.join('data','anchor')

In [6]:
os.makedirs(pos_path)
os.makedirs(neg_path)
os.makedirs(anchor_path)

In [8]:
!tar -xf lfw.tgz

In [12]:
#changing file location
for i in os.listdir('lfw'):
    for j in os.listdir(os.path.join('lfw',i)):
        oldpath=os.path.join('lfw',i,j)
        newpath=os.path.join(neg_path,j)
        os.replace(oldpath,newpath)

In [10]:
capture = cv2.VideoCapture(0)
while capture.isOpened(): 
    ret, frame = capture.read()
    #Cut down frame
    frame=frame[120:370,200:450,:]
    
    cv2.imshow("image",frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('a'):
        imgname=os.path.join(anchor_path,'{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname,frame)
    elif key == ord('p'):
        imgname=os.path.join(pos_path,'{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname,frame)
    elif key == ord('e'):
        break

capture.release()
cv2.destroyAllWindows()



QObject::moveToThread: Current thread (0x52560b0) is not the object's thread (0x3e61120).
QObject::moveToThread: Current thread (0x52560b0) is not the object's thread (0x3e61120).



In [8]:
anchor=tf.data.Dataset.list_files(anchor_path+'/*.jpg').take(350)
positive=tf.data.Dataset.list_files(pos_path+'/*.jpg').take(350)
negative=tf.data.Dataset.list_files(neg_path+'/*.jpg').take(350)

2023-04-09 23:37:46.168300: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/priyanshu/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-04-09 23:37:46.168341: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-09 23:37:46.168407: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (priyanshu-Inspiron-7400): /proc/driver/nvidia/version does not exist
2023-04-09 23:37:46.169173: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
a=anchor.as_numpy_iterator()
a.next()

b'data/anchor/c42e30b6-d606-11ed-b7da-6fbc72172f5b.jpg'

In [10]:
def preprocess(fpath):
    
    byte_img = tf.io.read_file(fpath)
    img = tf.io.decode_jpeg(byte_img)
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (105,105))
    # Scale image to be between 0 and 1 
    img = img / 255.0
    return img

In [11]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)  #creating our dataset of negatives and positives

In [12]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [13]:
#data loader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [21]:
# Training partition
training_data = data.take(round(len(data)*.75))
training_data = training_data.batch(16)
training_data = training_data.prefetch(8)

In [24]:
# Testing partition
testing_data = data.skip(round(len(data)*.75))
testing_data = testing_data.take(round(len(data)*.25))
testing_data = testing_data.batch(16)
testing_data = testing_data.prefetch(8)